<a href="https://colab.research.google.com/github/amitca71/colab/blob/main/llama3_text2cypher_chat_unsloth_chang_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [73]:
from google.colab import drive
import os,subprocess
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
env_file = f'{root_dir}/Colab Notebooks/env'
def get_env_vars(conf_file_path):
  variables={}
  command = f'cat "{env_file}"'
  output, error =subprocess.Popen(command, stdout=subprocess.PIPE, shell=True, executable='/bin/bash').communicate()

  if output:
      for line in output.decode().strip().replace('"', '').splitlines():
          key, _, value = line.partition("=")
          key=key.replace("export", "").strip()
          if(key!=""):
            variables[key]=value
            os.environ[key] = value
#            print(key, os.getenv(key))
  return(variables)
env_vars=get_env_vars(env_file)
hf_token=env_vars['HUGGINGFACE_TOKEN']
github_token=env_vars['GITHUB_TOKEN']


Mounted at /content/gdrive


In [1]:
%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes
!pip install torch --upgrade

In [4]:
#parameters
base_model="phi-3"
model_name="unsloth/Phi-3-mini-4k-instruct"
hf_dataset="tomasonjo/text2cypher-gpt4o-clean"

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
#    model_name = "unsloth/llama-3-8b-Instruct-bnb-4bit",
#"unsloth/Phi-3-mini-4k-instruct"

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


config.json:   0%|          | 0.00/623 [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Mistral patching release 2024.7
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors.index.json:   0%|          | 0.00/29.9k [00:00<?, ?B/s]

model-00001-of-00006.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.7 patched 40 layers with 40 QKV layers, 40 O layers and 40 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the `ChatML` format for conversation style finetunes. We use [Open Assistant conversations](https://huggingface.co/datasets/philschmid/guanaco-sharegpt-style) in ShareGPT style. ChatML renders multi turn conversations like below:

```
<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
What's the capital of France?<|im_end|>
<|im_start|>assistant
Paris.
```

Let's see how the `ChatML` format works by printing the 5th element

In [6]:
from unsloth.chat_templates import get_chat_template
from datasets import load_dataset

tokenizer = get_chat_template(
    tokenizer,
    chat_template = base_model, # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth

)

def formatting_prompts_func(examples):
    texts = []
    for question, cypher, schema in zip(examples['question'], examples['cypher'], examples['schema']):
        convo = [
            {"role": "system", "content": "Given an input question, convert it to a Cypher query. No pre-amble."},
            {"role": "user", "content": f"""Based on the Neo4j graph schema below, write a Cypher query that would answer the user's question:
{schema}

Question: {question}
Cypher query:"""},
            {"role": "assistant", "content": f"{cypher}"}
        ]
        text = tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False)
        texts.append(text)
    return { "text" : texts }


dataset = load_dataset(hf_dataset, split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split:   0%|          | 0/7735 [00:00<?, ? examples/s]

Map:   0%|          | 0/7735 [00:00<?, ? examples/s]

In [7]:
dataset[5]["text"]

'<s><|system|>\nGiven an input question, convert it to a Cypher query. No pre-amble.<|end|>\n<|user|>\nBased on the Neo4j graph schema below, write a Cypher query that would answer the user\'s question:\nNode properties:\n- **User**\n  - `label`: STRING Example: "inchristbl.bsky.social"\n  - `key`: STRING Example: "0"\n  - `y`: FLOAT Example: "6000.659"\n  - `color`: STRING Example: "#3CB371"\n  - `x`: FLOAT Example: "-4383.217"\n  - `area`: FLOAT Example: "2.6"\n  - `size`: FLOAT Example: "1.82"\nRelationship properties:\n- **INTERACTED**\n  - `weight: INTEGER` Example: "1"\n  - `ogWeight: INTEGER` Example: "1"\n  - `size: FLOAT` Example: "0.2"\n  - `key: STRING` Example: "geid_68_0"\nThe relationships:\n(:User)-[:INTERACTED]->(:User)\n\nQuestion: Who are the top 5 users with the most interactions having a weight greater than 5?\nCypher query:<|end|>\n<|assistant|>\nMATCH (u1:User)-[r:INTERACTED]->(u2:User)\nWHERE r.weight > 5\nWITH u1, COUNT(r) AS interactionCount\nORDER BY interacti

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [9]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/7735 [00:00<?, ? examples/s]

RuntimeError: One of the subprocesses has abruptly died during map operation.To debug the error, disable multiprocessing.

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [ ]:
trainer_stats = trainer.train()

In [9]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
#used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

NameError: name 'start_gpu_memory' is not defined

<a name="Inference"></a>
### Inference
Let's run the model! Since we're using `ChatML`, use `apply_chat_template` with `add_generation_prompt` set to `True` for inference.

In [23]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = base_model, # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    map_eos_token = True, # Maps <|im_end|> to </s> instead
#    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"},
)

FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
      {"role": "system", "content": "Given an input question, convert it to a Cypher query. No pre-amble."},
      {"role": "user", "content": f"""Based on the Neo4j graph schema below, write a Cypher query that would answer the user's question:
{dataset[0]['schema']}

Question: {dataset[0]['question']}
Cypher query:"""}
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<s><|system|> Given an input question, convert it to a Cypher query. No pre-amble.<|end|><|user|> Based on the Neo4j graph schema below, write a Cypher query that would answer the user\'s question:\nNode properties:\n- **User**\n  - `label`: STRING Example: "inchristbl.bsky.social"\n  - `key`: STRING Example: "0"\n  - `y`: FLOAT Example: "6000.659"\n  - `color`: STRING Example: "#3CB371"\n  - `x`: FLOAT Example: "-4383.217"\n  - `area`: FLOAT Example: "2.6"\n  - `size`: FLOAT Example: "1.82"\nRelationship properties:\n- **INTERACTED**\n  - `weight: INTEGER` Example: "1"\n  - `ogWeight: INTEGER` Example: "1"\n  - `size: FLOAT` Example: "0.2"\n  - `key: STRING` Example: "geid_68_0"\nThe relationships:\n(:User)-[:INTERACTED]->(:User)\n\nQuestion: Find the top 3 users interacted with by \'sarahp.bsky.social\'.\nCypher query:<|end|><|assistant|> MATCH (u1:User {label: \'sarahp.bsky.social\'})-[:INTERACTED]->(u2:User)\nRETURN u2.label AS user, COUNT(*) AS interactions\nORDER BY interaction

In [31]:
messages

[{'from': 'human', 'value': 'What is a famous tall tower in Paris?'}]

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [11]:
model.save_pretrained("lora_model") # Local saving
saved_model_name="amitca71/lora_model_text2cypher_" + model_name.replace("/", "_")
model.push_to_hub(saved_model_name, token = hf_token) # Online saving

README.md:   0%|          | 0.00/585 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/120M [00:00<?, ?B/s]

Saved model to https://huggingface.co/amitca71/lora_model_text2cypher_unsloth_phi-3-mini-4k-instruct


Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [32]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = saved_model_name, # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "human", "content": "What is a famous tall tower in Paris?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128, use_cache = True)

<s> <|human|>
What is a famous tall tower in Paris?<|end|><|assistant|> The Eiffel Tower is a famous tall tower in Paris, France. It was designed by Gustave Eiffel and completed in 1889 for the Exposition Universelle, a world's fair marking the centennial celebration of the French Revolution. Standing at 300 meters (984 feet) tall, it was the tallest man-made structure in the world until the completion of the Chrysler Building in New York in 1930.<|end|><|endoftext|>


### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [34]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = hf_token)

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = hf_token)

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = hf_token)

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

In [35]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = hf_token)

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = hf_token)

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = hf_token)

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in `llama.cpp` or a UI based system like `GPT4All`. You can install GPT4All by going [here](https://gpt4all.io/index.html).